# TF-IDF & Vector Space Model

## Apa itu VSM (Vector Space Model)

Vector Space Model (VSM) adalah kerangka kerja matriks yang digunakan dalam temu kembali informasi dan pemrosesan bahasa alami (NLP) untuk merepresentasikan dan menganalisis data visual. VSM sangat penting dalam penggalian teks, pencarian dokumen, dan tugas-tugas pembelajaran mesin berbasis teks seperti klasifikasi dokumen, pencarian informasi, dan analisis kemiripan teks.

![image.png](https://i0.wp.com/spotintelligence.com/wp-content/uploads/2023/09/vector-space-model.jpg?resize=960%2C540&ssl=1)

Setiap dimensi berhubungan dengan term yang unik, sementara dokumen dan query dapat direpresentasikan sebagai vektor di dalam ruang tersebut.

## Term Frequency-Inverse Document Frequency (TF-IDF)

TF-IDF adalah singkatan dari Term Frequency Inverse Document Frequency. Hal ini dapat didefinisikan sebagai perhitungan seberapa relevan sebuah kata dalam kumpulan atau corpus terhadap sebuah teks. Nilai relevansi meningkat secara relatif terhadap berapa kali sebuah kata muncul di dalam teks, namun dikompensasi oleh frekuensi kata di dalam corpus (kumpulan data).

## Term Frequency

Pada dokumen d, frekuensi merepresentasikan jumlah kemunculan kata t. Oleh karena itu, kita dapat melihat bahwa frekuensi akan menjadi lebih relevan ketika sebuah kata muncul dalam teks, yang mana hal ini bersifat relatif. Karena urutan istilah tidak signifikan, kita dapat menggunakan vektor untuk mendeskripsikan teks dalam kumpulan model term. Untuk setiap istilah tertentu dalam teks, ada sebuah entri dengan nilai yang merupakan frekuensi term.

**tf(t,d) = count of t in d / number of words in d**

## Inverse Document Frequency

Pada dasarnya, ini menguji seberapa relevan kata tersebut. Tujuan utama dari pencarian ini adalah untuk menemukan dokumen yang sesuai dengan pencarian. Karena tf menganggap semua istilah sama pentingnya, maka frekuensi term tidak hanya dapat digunakan untuk mengukur bobot term dalam dokumen. Pertama, cari frekuensi dokumen dari suatu istilah t dengan menghitung jumlah dokumen yang mengandung term tersebut:

**idf(t) = log(N/ df(t))**

Penjelasan:<br>
**df(t)** = **N(t)**<br>
dimana<br>
**df(t)** = **Document frequency of a term t**<br>
**N(t)** = **Number of documents containing the term t**<br>

## TF-IDF (Term Frequency-Inverse Document Frequency)

**tf-idf(t, d) = tf(t, d) * idf(t)**

## Proses TF-IDF dan menjadikannya VSM (Vector Space Model)

### Import Library/Tool yang dibutuhkan

In [43]:
# Library untuk data manipulation
import pandas as pd
from tqdm import tqdm
import re
import string

# Library untuk text preprocessing
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

# Library untuk text vectorization/TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to C:\Users\LAB
[nltk_data]     SISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\LAB
[nltk_data]     SISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


* **pandas** digunakan untuk membuat dataframe agar mudah dibaca.
* **tqdm** Untuk mentracking proses program
* **re** (regular expression) digunakan untuk mengenali pola kata/kalimat
* **nltk** (natural language toolkit) berfungsi untuk melakukan proses yang berkaitan dengan bahasa (teks).
* **sklearn** digunakan dalam pemrosesan data untuk kebutuhan machine learning atau data science. Dalam tugas ini, menggunakan TfidfTransformer untuk menghitung TF-IDF
* **sastrawi** berfungsi untuk membersihkan mengurangi kata-kata imbuhan menjadi kata basic (sastrawi untuk stemming bahasa Indonesia)

### Import data berita CSV

In [44]:
data = pd.read_csv("data_100.csv")
data.columns = data.columns.str.strip()
data

,judul,isi,tanggal,kategori
0,"Putin Dukung Kamala Harris: Tawanya Menular, A...",Presiden Rusia Vladimir Putinbergurau bahwa di...,"Jumat, 06 Sep 2024 14:31 WIB",Internasional
1,"Tinggalkan RI, Paus Fransiskus Twit Perpisahan...",Pemimpin Gereja Katolik seluruh dunia Paus Fra...,"Jumat, 06 Sep 2024 14:01 WIB",Internasional
2,"Pamit ke Menag Yaqut di Bandara, Paus Fransisk...",Pemimpin Gereja Katolik seluruh dunia Paus Fra...,"Jumat, 06 Sep 2024 13:30 WIB",Internasional
3,"FOTO: Sampai Jumpa Lagi, Paus Fransiskus ...",...,"Jumat, 06 Sep 2024 13:00 WIB",Internasional
4,Paus Fransiskus Pilih Nasi Goreng Jadi Menu di...,"Pemimpin Gereja Katolik seluruh dunia, Paus Fr...","Jumat, 06 Sep 2024 12:24 WIB",Internasional
...,...,...,...,...
95,Rekayasa Lalin saat Paus Fransiskus Misa Akbar...,Kepolisian melakukan rekayasa lalu lintas di s...,"Kamis, 05 Sep 2024 07:45 WIB",Nasional
96,VIDEO: Kaesang Bungkam saat Ditanya soal Jet P...,Ketua Umum PSI Kaesang Pangarep irit bicara sa...,"Kamis, 05 Sep 2024 07:33 WIB",Nasional
97,"Misa Akbar Paus di GBK Hari Ini, Pemprov Imbau...",Penjabat (Pj) DKI JakartaHeru Budi Hartono men...,"Kamis, 05 Sep 2024 07:19 WIB",Nasional
98,"Nyaris Adu Jotos dengan Silfester Matutina, Ch...",Juru Bicara PDIPChico Hakim menjelaskan soal m...,"Kamis, 05 Sep 2024 07:15 WIB",Nasional


Import data berita csv yang telah disimpan sebelumnya.<br>
dan membersihkan whitespace pada col header

### Fungsi clean_text()

In [45]:
def clean_text(text):
	text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', ' ', text) # Menghapus https* and www*
	text = re.sub(r'@[^\s]+', ' ', text) # Menghapus username
	text = re.sub(r'[\s]+', ' ', text) # Menghapus tambahan spasi
	text = re.sub(r'#([^\s]+)', ' ', text) # Menghapus hashtags
	text = re.sub(r'rt', ' ', text) # Menghapus retweet
	text = text.translate(str.maketrans("","",string.punctuation)) # Menghapus tanda baca
	text = re.sub(r'\d', ' ', text) # Menghapus angka
	text = text.lower()
	text = text.encode('ascii','ignore').decode('utf-8') #Menghapus ASCII dan unicode
	text = re.sub(r'[^\x00-\x7f]',r'', text)
	text = text.replace('\n','') #Menghapus baris baru
	text = text.strip()
	return text

Fungsi ini untuk membersihkan text, lebih tepatnya membersihkan teks seperti; menghapus hastag, unicode, dsb

### Fungsi stemming_indo()

In [46]:
def stemming_indo(text):
	factory = StemmerFactory()
	stemmer = factory.create_stemmer()
	text = stemmer.stem(text)
	return text

Fungsi ini digunakan untuk menstemming atau membersihkan kata seperti:

katanya = kata<br>
menggunakan = guna

### Fungsi clean_stopword()

In [47]:
def clean_stopword(tokens):
	listStopword =  set(stopwords.words('indonesian'))
	removed = []
	for t in tokens:
		if t not in listStopword:
			removed.append(t)
	return removed

Fungsi untuk membuang kata yang tidak digunakan seperti:

di, dan, dsb

### Proses preprosesing text setiap dokumen

In [48]:
content = data['isi']
all_docs = []
for doc in tqdm(content):
	cleaned_doc = clean_text(doc)
	stemmed_doc = stemming_indo(cleaned_doc)
	tokens = nltk.tokenize.word_tokenize(stemmed_doc)
	cleaned_tokens = clean_stopword(tokens)
	joined_strings = []
	all_docs.append(' '.join(cleaned_tokens))
print(all_docs)

100%|██████████| 100/100 [00:02<00:00, 36.83it/s]

['presiden rusia vladimir putinbergurau dukung calon presiden capres amerika serikat pa ai demokrat kamala harris alihalih capres pa ai republik donald trump putin kelakar harris milik tawa tular cermin dunia baikbaik te awa ekspresif tular bera i baikbaik putin sepe i kutip reuters putin lontar canda pe anyaan moderator acara forum ekonomi rusia kamis sang moderator be anya pandang putin kena pilih as putin serius pilih as gantung pilih rakyat amerika kelakar presiden tahana joe biden dukung harris pilih harris laku dukung putin putin komentar pilih biden banding trump biden jadul mudah tebak badan intelijen as percaya putin tuju trump menang pilih presiden pilpres harris trump dukung ukraina perang kremlin', 'pimpin gereja katolik dunia paus fransiskusmenyampaikan pesan pisah bahasa indonesia jelang berangkat papua nugini negara keduayang kunjung jalan apostolik jumat pesan pisah risi terima kasih kesan kunjung indonesia september terima kasih sambut paus x moga tuhan kati be umbuh b

Proses untuk mempersiapkan teks pada setiap dokumen yang diproses menggunakan fungsi-fungsi yang sudah dibuat sebelumnya, yang selanjutnya akan di tf-idf dan vsm

### Proses TF-IDF dan pembuatan VSM

In [49]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(all_docs)
feature_names = tfidf.get_feature_names_out()
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
df_tfidf.insert(0, 'doc', [f'doc{i+1}' for i in range(len(content))])

df_tfidf

,doc,aa,aaditemukan,aamiin,abai,abalos,abang,abar,abat,abd,...,zahro,zaini,zakaria,zaky,zat,zenix,zero,zirah,zoom,zulkarnain
0,doc1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,doc2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,doc3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,doc4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,doc5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,doc96,0.0,0.0,0.0,0.0,0.0,0.089077,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,doc97,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,doc98,0.0,0.0,0.0,0.0,0.0,0.069371,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,doc99,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Proses untuk membuat TF-IDF dan membentuk VSM dalam dataframe